In [1]:
import os
import pyriemann
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from scipy import stats
from sklearn.model_selection import cross_val_score
from scipy.fftpack import fft
from scipy import signal

#from mne.preprocessing import Xdawn
from pyriemann.estimation import XdawnCovariances, Xdawn
from pyriemann.estimation import ERPCovariances
from pyriemann.tangentspace import TangentSpace
from sklearn.linear_model import LogisticRegression
import warnings
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
#from mne.decoding import Vectorizer
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis as QDA
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.neighbors import KNeighborsClassifier
from pyriemann.classification import MDM
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
#import mne

### Import data

Run the first cell if you want to only look at individuals data\
and the second cell if you want a combined data set

In [4]:
#Combined test subjects and sessions
subjects = ("","FnF_amin")
sessions = ("1","2","3")
data_dir = Path("../timeflux/data/")
paths = np.array([[data_dir / f"{subj}" / f"{sesh}" for sesh in sessions] for subj in subjects]).flatten()
numSess = len(sessions)*(len(subjects)-1)
x_data = np.empty((0,4,256)) # Placeholder for data
y_data = np.empty((0)) # Placeholder for data
for path in paths:
    for file in path.glob("data_*.npz"):
        with open(file, "rb") as npz:
            archive = np.load(npz, allow_pickle=True)
            x_data = np.concatenate((x_data, archive["X"]))
            y_data = np.concatenate((y_data, archive["y"]))

def objmap(n):
    if (n=="face"):
        return 1
    else:
        return 0
def FNFmap(n):
    if (n=="familiar"):
        return 1
    else:
        return 0
#y_data= np.array([objmap(el) for el in y_data])  #obj
y_data= np.array([FNFmap(el) for el in y_data])  #FNF
#y_data = y_data.astype(int)
print(np.shape(x_data))

(150, 4, 256)


### Cross-sessions eval

In [5]:
class Vectorizer(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass

    def fit(self, X, y):
        """fit."""
        return self

    def transform(self, X):
        """transform. """
        return np.reshape(X, (X.shape[0], -1))

In [6]:

pipelines = {}

pipelines["XDAWN covariances + Logistic regression "] = make_pipeline(
    XdawnCovariances(4, estimator="lwf", xdawn_estimator="scm"),
    TangentSpace(metric="riemann"),
    StandardScaler(),
    LogisticRegression(),
)

pipelines["XDAWN + LDA"] = make_pipeline(
    Xdawn(nfilter=4, estimator="scm"),
    Vectorizer(),
    StandardScaler(),
    LDA(solver="lsqr", shrinkage="auto"),
)

pipelines["LDA"] = make_pipeline(
    Vectorizer(),
    StandardScaler(),
    LDA(solver="lsqr", shrinkage="auto"),
)



#pipelines["QDA"] = make_pipeline(
#    Vectorizer(),
#    StandardScaler(),
#    QDA(),
#)


pipelines["XDAWN covariances + LDA"] = make_pipeline(
    XdawnCovariances(4, estimator="lwf", xdawn_estimator="scm"),
    TangentSpace(metric="riemann"),
    StandardScaler(),
    LDA(solver="lsqr", shrinkage="auto"),
)
pipelines["XDAWN covariances + SVM"] = make_pipeline(
    XdawnCovariances(4, estimator="lwf", xdawn_estimator="scm"),
    TangentSpace(metric="riemann"),
    StandardScaler(),
    SVC(),
)

#pipelines["XDAWN covariances + QDA"] = make_pipeline(
#    XdawnCovariances(4, estimator="lwf", xdawn_estimator="scm"),
#    TangentSpace(metric="riemann"),
#    StandardScaler(),
#    QDA(),
#)

pipelines["XDAWN covariances + KNearestNeighbors"] = make_pipeline(
    XdawnCovariances(3, estimator="lwf", xdawn_estimator="scm"),
    TangentSpace(metric="riemann"),
    KNeighborsClassifier(),
)
pipelines["XDAWN covariances + MDM"]=make_pipeline(
    XdawnCovariances(3, estimator="lwf", xdawn_estimator="scm"),
    MDM(),
)

pipelines["MLP"] = make_pipeline(
    Vectorizer(),
    StandardScaler(),
    MLPClassifier(hidden_layer_sizes=(10,10,2), max_iter=30000,tol=10**(-8),activation = 'tanh',solver='adam',random_state=123)
)
pipelines["XDAWN covariances + MLP "]= make_pipeline(
    XdawnCovariances(4, estimator="lwf", xdawn_estimator="scm"),
    TangentSpace(metric="riemann"),
    Vectorizer(),
    StandardScaler(),
    MLPClassifier(hidden_layer_sizes=(20,20,10,2), max_iter=30000,tol=10**(-8),activation = 'tanh',solver='adam',random_state=123)
)

### Within-Session Evaluation

In [7]:
for name, pipe in pipelines.items():
    
    for i in range(numSess):
        cv = KFold(50)
        x_split=x_data[50*i:50*(i+1)]
        y_split=y_data[50*i:50*(i+1)]
        preds = np.zeros(50)
        for train_idx, test_idx in cv.split(x_split):
            
            y_train, y_test = y_split[train_idx], y_split[test_idx]

            pipe.fit(x_split[train_idx], y_train)
            preds[test_idx] = pipe.predict(x_split[test_idx])

        # Printing the results
        acc = np.mean(preds == y_split)
        print(name + ":")
        print("Classification accuracy: %f " % (acc))
        
        #cm = confusion_matrix(y_data, preds)
        #ConfusionMatrixDisplay(cm).plot()
        #plt.show()

XDAWN covariances + Logistic regression :
Classification accuracy: 0.600000 
XDAWN covariances + Logistic regression :
Classification accuracy: 0.640000 
XDAWN covariances + Logistic regression :
Classification accuracy: 0.480000 


KeyboardInterrupt: 

### Cross-Session Evaluation:

In [ ]:


cv = KFold(numSess)
preds = np.zeros(len(y_data))
for name, pipe in pipelines.items():
    for train_idx, test_idx in cv.split(x_data):
        y_train, y_test = y_data[train_idx], y_data[test_idx]

        pipe.fit(x_data[train_idx], y_train)
        preds[test_idx] = pipe.predict(x_data[test_idx])

    # Printing the results
    acc = np.mean(preds == y_data)
    print(name + ", classification accuracy: %f " % (acc))
    
#    cm = confusion_matrix(y_data, preds)
#    ConfusionMatrixDisplay(cm).plot()
 #   plt.show()

XDAWN covariances + Logistic regression , classification accuracy: 0.566667 
XDAWN + LDA, classification accuracy: 0.493333 
LDA, classification accuracy: 0.540000 
XDAWN covariances + LDA, classification accuracy: 0.573333 
XDAWN covariances + SVM, classification accuracy: 0.526667 
XDAWN covariances + KNearestNeighbors, classification accuracy: 0.526667 
XDAWN covariances + MDM, classification accuracy: 0.486667 
MLP, classification accuracy: 0.533333 


In [15]:
#for train_idx, test_idx in cv.split(x_data):
    y_train, y_test = y_data[train_idx], y_data[test_idx]
    XDC = XdawnCovariances(4, estimator="lwf", xdawn_estimator="scm")
    x_train = x_data[train_idx]
    covar = 100000000*XDC.fit_transform(x_train,y_train)
    print(np.shape(covar))   
    print(np.shape(x_train))      
    
    
    x_train= np.concatenate((x_train.reshape((100, 1024)),covar.reshape(100,256)),axis=1)
    print(np.shape(x_train)) 
    XdcMlpPipe.fit(x_data[train_idx], y_train)
    preds[test_idx] = XdcMlpPipe.predict(x_data[test_idx])

# Printing the results
acc = np.mean(preds == y_data)
print("Classification accuracy: %f " % (acc))

cm = confusion_matrix(y_data, preds)
ConfusionMatrixDisplay(cm).plot()
plt.show()

IndentationError: unexpected indent (1634055813.py, line 2)

### Cross-Subject Evaluation

In [16]:
cv = KFold(2)
preds = np.zeros(len(y_data))
for name, pipe in pipelines.items():
    for train_idx, test_idx in cv.split(x_data):
        y_train, y_test = y_data[train_idx], y_data[test_idx]

        pipe.fit(x_data[train_idx], y_train)
        preds[test_idx] = pipe.predict(x_data[test_idx])

    # Printing the results
    acc = np.mean(preds == y_data)
    print(name + ", classification accuracy: %f " % (acc))

XDAWN covariances + Logistic regression , classification accuracy: 0.520000 
XDAWN + LDA, classification accuracy: 0.530000 
LDA, classification accuracy: 0.533333 
XDAWN covariances + LDA, classification accuracy: 0.523333 
XDAWN covariances + SVM, classification accuracy: 0.550000 
XDAWN covariances + SVM2, classification accuracy: 0.503333 
XDAWN covariances + SVM3, classification accuracy: 0.486667 
XDAWN covariances + KNearestNeighbors, classification accuracy: 0.520000 
XDAWN covariances + MDM, classification accuracy: 0.503333 
MLP, classification accuracy: 0.526667 
XDAWN covariances + MLP , classification accuracy: 0.550000 
